In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import sys
from IPython.display import HTML
from matplotlib import pyplot as plt
from celluloid import Camera
import plotly.graph_objects as go

import networkx as nx

from IPython.display import HTML
from matplotlib import pyplot as plt
from celluloid import Camera
import collections


import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
import ast

In [2]:
emails_df = pd.read_csv('Data/emails_tokenized.csv')
employee_df = pd.read_csv('Data/employees.csv')
network_df = pd.read_csv('Data/network.csv')

In [3]:
emails_df = emails_df[~emails_df['token_subject'].isna()]

exec_list = ['CEO','President','Vice President']

job_count = collections.Counter()
for rank in employee_df['job_rank']:
    job_count[rank] +=1

job_count

Counter({nan: 51,
         'Manager': 12,
         'Employee': 34,
         'Vice President': 21,
         'Director': 12,
         'Trader': 11,
         'President': 4,
         'CEO': 3,
         'Managing Director': 2,
         'In House Lawyer': 1})

In [4]:
stems = ['fw','draft','p','2001']

emails_df['token_subject']= [ast.literal_eval(x) for x in emails_df['token_subject']]
subjects = []
for email in emails_df['token_subject']:
    for stem in stems:
        while stem in email:
            email.remove(stem)
    subjects.append(email)
emails_df['token_subject'] = subjects

In [5]:
emails_df.head()

,Unnamed: 0,mid,sender,date,message_id,subject,body,folder,cleaned_body,cleaned_subject,sender_level,token_body,token_subject
0,24,76,mary.hain@enron.com,2000-08-17 07:11:00,<5722839.1075863606028.JavaMail.evans@thyme>,Enron s transmission/power exchange model for ...,---------------------- Forwarded by Mary Hain/...,Robert_Badeer_Aug2000Notes FoldersNotes inbox,enron s transmission/power exchange model for...,enron s transmission/power exchange model for ...,NaN,"[['enron', 'transmission/power', 'exchange', '...","[enron, transmission/power, exchange, model, d..."
1,41,93,cooper.richey@enron.com,2000-08-23 04:39:00,<4083763.1075863605647.JavaMail.evans@thyme>,Change to EnData,The Fundamentals Group is moving Database serv...,Robert_Badeer_Aug2000Notes FoldersNotes inbox,the fundamentals group is moving database serv...,change to endata,Manager,"[['fundamentals', 'group', 'moving', 'database...","[change, endata]"
2,107,159,mary.hain@enron.com,2000-08-29 06:28:00,<14457348.1075863603979.JavaMail.evans@thyme>,ISO To Participate in Super Peak Market,FYI---------------------- Forwarded by Mary Ha...,Robert_Badeer_Aug2000Notes FoldersNotes inbox,"iso to participate in super peak marketfolks,...",iso to participate in super peak et,NaN,"[['iso', 'participate', 'super', 'peak', 'mark...","[iso, participate, super, peak, et]"
3,137,465,jeff.dasovich@enron.com,2000-08-07 12:23:00,<769016.1075863596202.JavaMail.evans@thyme>,CalPX Market Compliance Unit Submits its 2nd A...,FYI. The report posted on the website fails t...,Robert_Badeer_Aug2000Notes FoldersCapx,calpx market compliance unit submits its 2nd ...,calpx et compliance unit submits its 2nd annu...,Employee,"[['calpx', 'market', 'compliance', 'unit', 'su...","[calpx, et, compliance, unit, submits, 2nd, an..."
4,189,517,jeff.dasovich@enron.com,2000-08-10 03:12:00,<28494229.1075863594884.JavaMail.evans@thyme>,Re: Governor s Press Conference -- Shot Across...,There appears to be a pattern forming. The ad...,Robert_Badeer_Aug2000Notes FoldersCalifornia,governor s press conference -- shot across th...,re: governor s press conference -- shot across...,Employee,"[['governor', 'press', 'conference', '--', 'sh...","[governor, press, conference, --, shot, across..."


In [6]:
emails_df['token_body']= [ast.literal_eval(x) for x in emails_df['token_body']]
subjects = []
for email in emails_df['token_body']:
    sents = []
    for sent in email:
        for stem in stems:
            while stem in sent:
                sent.remove(stem)
        sents.append(email)
    subjects.append(sents)
emails_df['token_body'] = subjects

In [7]:
subject_tokens = list(emails_df['token_subject']) 

In [9]:
dictionary = gensim.corpora.Dictionary(subject_tokens)

In [10]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 discussion
1 enron
2 exchange
3 model
4 transmission/power
5 change
6 endata
7 et
8 iso
9 participate
10 peak


In [11]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [12]:
bow_corpus = [dictionary.doc2bow(doc) for doc in subject_tokens]
bow_corpus[4310]

[(421, 1), (474, 1), (820, 1)]

In [13]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5546227871296954),
 (1, 0.3020901308523857),
 (2, 0.5623506333794593),
 (3, 0.5337573249856136)]


In [14]:
topic_num = 5
show_feature = 7

In [15]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=topic_num, id2word=dictionary, passes=20, workers=2)

In [16]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=topic_num, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.028*"deal" + 0.025*"meeting" + 0.012*"hello" + 0.010*"netco" + 0.009*"information" + 0.008*"enron" + 0.008*"isda" + 0.008*"good" + 0.007*"morning" + 0.007*"dinner"
Topic: 1 Word: 0.020*"agreement" + 0.016*"contract" + 0.016*"call" + 0.014*"hey" + 0.011*"tw" + 0.011*"conference" + 0.009*"book" + 0.009*"approval" + 0.009*"contracts" + 0.008*"report"
Topic: 2 Word: 0.019*"power" + 0.014*"gas" + 0.014*"master" + 0.013*"energy" + 0.012*"agreement" + 0.011*"info" + 0.010*"eol" + 0.010*"project" + 0.009*"inc" + 0.008*"ces"
Topic: 3 Word: 0.017*"lunch" + 0.016*"update" + 0.013*"..." + 0.012*"gisb" + 0.012*"letter" + 0.010*"checkout" + 0.010*"ca" + 0.009*"test" + 0.008*"e" + 0.008*"schedule"
Topic: 4 Word: 0.015*"new" + 0.014*"enron" + 0.012*"credit" + 0.012*"california" + 0.011*"hi" + 0.009*"game" + 0.009*"fwd" + 0.009*"capacity" + 0.009*"eol" + 0.008*"resume"


In [17]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, show_feature)))


Score: 0.5496178865432739	 
Topic: 0.053*"gas" + 0.033*"deal" + 0.030*"agreement" + 0.020*"report" + 0.020*"california" + 0.017*"list" + 0.016*"capacity"

Score: 0.30000898241996765	 
Topic: 0.037*"energy" + 0.024*"letter" + 0.021*"eol" + 0.020*"trading" + 0.019*"enron" + 0.018*"agreement" + 0.018*"credit"

Score: 0.05036928504705429	 
Topic: 0.065*"meeting" + 0.025*"..." + 0.022*"call" + 0.019*"conference" + 0.012*"lunch" + 0.011*"game" + 0.010*"management"

Score: 0.050001926720142365	 
Topic: 0.034*"update" + 0.025*"contract" + 0.015*"revised" + 0.015*"--" + 0.013*"dinner" + 0.013*"e" + 0.012*"final"

Score: 0.05000191181898117	 
Topic: 0.056*"new" + 0.034*"enron" + 0.017*"deals" + 0.016*"schedule" + 0.015*"power" + 0.012*"information" + 0.011*"book"


In [18]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, show_feature)))


Score: 0.7407269477844238	 
Topic: 0.019*"power" + 0.014*"gas" + 0.014*"master" + 0.013*"energy" + 0.012*"agreement" + 0.011*"info" + 0.010*"eol"

Score: 0.1052582785487175	 
Topic: 0.015*"new" + 0.014*"enron" + 0.012*"credit" + 0.012*"california" + 0.011*"hi" + 0.009*"game" + 0.009*"fwd"

Score: 0.05215754359960556	 
Topic: 0.017*"lunch" + 0.016*"update" + 0.013*"..." + 0.012*"gisb" + 0.012*"letter" + 0.010*"checkout" + 0.010*"ca"

Score: 0.05109405145049095	 
Topic: 0.028*"deal" + 0.025*"meeting" + 0.012*"hello" + 0.010*"netco" + 0.009*"information" + 0.008*"enron" + 0.008*"isda"

Score: 0.05076313763856888	 
Topic: 0.020*"agreement" + 0.016*"contract" + 0.016*"call" + 0.014*"hey" + 0.011*"tw" + 0.011*"conference" + 0.009*"book"


In [19]:
unseen_document = ['how','pentagon','be']
bow_vector = dictionary.doc2bow(subject_tokens[0])
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, show_feature)))

Score: 0.4398180842399597	 Topic: 0.053*"gas" + 0.033*"deal" + 0.030*"agreement" + 0.020*"report" + 0.020*"california" + 0.017*"list" + 0.016*"capacity"
Score: 0.43873241543769836	 Topic: 0.037*"energy" + 0.024*"letter" + 0.021*"eol" + 0.020*"trading" + 0.019*"enron" + 0.018*"agreement" + 0.018*"credit"
Score: 0.041292134672403336	 Topic: 0.056*"new" + 0.034*"enron" + 0.017*"deals" + 0.016*"schedule" + 0.015*"power" + 0.012*"information" + 0.011*"book"
Score: 0.04015404358506203	 Topic: 0.065*"meeting" + 0.025*"..." + 0.022*"call" + 0.019*"conference" + 0.012*"lunch" + 0.011*"game" + 0.010*"management"
Score: 0.04000331461429596	 Topic: 0.034*"update" + 0.025*"contract" + 0.015*"revised" + 0.015*"--" + 0.013*"dinner" + 0.013*"e" + 0.012*"final"


In [ ]:
['CEO','President','Vice President','Director','Employee']

In [20]:
subject_topics = []
for subject in emails_df['token_subject']:
    bow_vector = dictionary.doc2bow(subject)
    subject_topics.append(dict(lda_model[bow_vector]))

KeyboardInterrupt: 

In [ ]:
lda_model.get_topic_terms(0, topn=10)

In [ ]:
exec_email_df = emails_df[[x in exec_list for x in emails_df['sender_level']]]
employee_email_df = emails_df[emails_df['sender_level']=='Employee']

Total_Emails = list(employee_email_df['token_subject'])+list(exec_email_df['token_subject'])

exec_dictionary = gensim.corpora.Dictionary(Total_Emails)
exec_dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

exec_corpus = [dictionary.doc2bow(doc) for doc in list(exec_email_df['token_subject'])]

employee_corpus =[dictionary.doc2bow(doc) for doc in list(employee_email_df['token_subject'])]


Exec_model = gensim.models.LdaMulticore(exec_corpus, num_topics=topic_num, id2word=dictionary, passes=20, workers=2)
Employee_model = gensim.models.LdaMulticore(employee_corpus, num_topics=topic_num, id2word=dictionary, passes=20, workers=2)

In [ ]:
mdiff, annotation = Exec_model.diff(Employee_model)

In [ ]:
mdiff

In [ ]:
Exec_model.top_topics(employee_corpus,topn=5)

In [ ]:
for index, score in sorted(lda_model_tfidf[exec_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, show_feature)))